In [25]:
from yahoo_fin import stock_info as si
from matplotlib import pyplot as plt 
import pandas_datareader as dr
import quandl
import pandas as pd
import numpy as np
from pandas_datareader import data as wb
import datetime
import yfinance as yf
from requests_html import HTMLSession
from sklearn.utils import shuffle
import numpy_financial as npf
from oauth2client.service_account import ServiceAccountCredentials
import httplib2
import apiclient.discovery
from time import sleep

In [26]:
LTGrowth = 0.02
LTdis = 0.08


# ВВОДИМ ДАННЫЕ

In [27]:
 
FILTER = 'sector' # country sector

SECTOR = 'Metals & Mining'
COUNTRY = 'HKG' # DEU  CAN  GBR  KOR  HKG  SGP


In [28]:
# ~~~~~~~~~~~~~~~~~~~~ GOOGLE SHEETS ~~~~~~~~~~~~~~~~~~~~~~~~

# Файл, полученный в Google Developer Console
CREDENTIALS_FILE = 'Seetzzz-1cb93f64d8d7.json'
# ID Google Sheets документа (можно взять из его URL)
spreadsheet_id = '1bfNJIgSEo9V5Jww1-EoUh_onba2bGY2LpDVx4aYlPzc'
spreadsheet_id2 = '1A3leW6ZfsoVEPXZsv0Loj4eAbyKRchnHrJLdP4RIXDA'

# Авторизуемся и получаем service — экземпляр доступа к API
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    CREDENTIALS_FILE,
    ['https://www.googleapis.com/auth/spreadsheets',
     'https://www.googleapis.com/auth/drive'])
httpAuth = credentials.authorize(httplib2.Http())
service = apiclient.discovery.build('sheets', 'v4', http=httpAuth)

In [29]:
long_term_growth = 0.01 # динамическое значение инфляции для каждой страны
iterations = 1000

In [30]:
def main_table_qan(COUNTRY, COMPANY):
    data_csv_ticker = pd.read_csv('TICKER_companies.csv')
    data_csv = data_csv_ticker.set_index('country')
    data_industry_sector = pd.read_excel('SECTORS.xlsx')
    data_ticker = data_csv[['ticker', 'compnumber', 'mic']]
    
    data_ticker.loc[COUNTRY]
    company_number = data_ticker.loc[COUNTRY].set_index('ticker').loc[COMPANY]['compnumber']

    company_number = data_ticker.loc[COUNTRY].set_index('ticker').loc[COMPANY]['compnumber']
    company_sector = data_ticker.loc[COUNTRY].set_index('ticker').loc[COMPANY]['mic'][0:3]
    company_sector_ticker = data_industry_sector.set_index('Industry_mic').loc[float(company_sector)][0]
    
    data_csv_ticker = pd.read_csv('TICKER_companies.csv')
    data_csv = data_csv_ticker.set_index('country')
    data_industry_sector = pd.read_excel('SECTORS.xlsx')

    data_ticker = data_csv[['ticker', 'compnumber', 'mic']]
    data_ticker['mic'].fillna('0.0')
    

    company_sector = data_ticker.loc[COUNTRY].set_index('ticker').loc[COMPANY]['mic'][0:3]
    company_sector_ticker = data_industry_sector.set_index('Industry_mic').loc[float(company_sector)][0]

    company_number = data_ticker.loc[COUNTRY].set_index('ticker').loc[COMPANY]['compnumber']
    company_sector = data_ticker.loc[COUNTRY].set_index('ticker').loc[COMPANY]['mic'][0:3]

    company_sector_ticker = data_industry_sector.set_index('Industry_mic').loc[float(company_sector)][0]

    
    quandl.ApiConfig.api_key = 'c3baa8FMfeFWP_wpbhKL'

    data = pd.DataFrame(quandl.get_table('MER/F1', compnumber=company_number, reporttype='A',  paginate=True))

    main_data = data.sort_values(by=['reportdate']).set_index('indicator')
    data_amount = main_data['amount']

    work_table = pd.DataFrame()

    work_table['date'] = main_data['reportdate'].T.drop_duplicates().T[::-1].reset_index(drop=True)[:10]


    work_table['revenue'], work_table['operatingIncome'], work_table['netIncome'], \
        work_table['totalAssets'], work_table['operatingMargin'], work_table['currentLiabilities'],\
        work_table['currentAssets'], work_table['stockholdersEquity'], work_table['shares'], \
        work_table['taxRate'], work_table['ROE'], work_table['cashandEquivalents'],  work_table['totalLiabilities'], \
        work_table['operatingCashFlow'], work_table['EPS'], \
        = data_amount['Total Revenue'][::-1].reset_index(drop=True)[:10], data_amount['Operating Income'][::-1].reset_index(drop=True)[:10],\
          data_amount['Net Income'][::-1].reset_index(drop=True)[:10],  \
          data_amount['Total Assets'][::-1].reset_index(drop=True)[:10], \
          data_amount['Operating Margin'][::-1].reset_index(drop=True)[:10], data_amount['Current Liabilities'][::-1].reset_index(drop=True)[:10], \
          data_amount['Current Assets'][::-1].reset_index(drop=True)[:10], \
          data_amount['''Stockholders' Equity'''][::-1].reset_index(drop=True)[:10], data_amount['Shares Outstanding'][::-1].reset_index(drop=True)[:10],\
          data_amount['Calculated Tax Rate'][::-1].reset_index(drop=True)[:10], data_amount['Return on Equity'][::-1].reset_index(drop=True)[:10],\
          data_amount['Cash and Equivalents'][::-1].reset_index(drop=True)[:10], \
          data_amount['Total Liabilities'][::-1].reset_index(drop=True)[:10], data_amount['Cash From Operations'][::-1].reset_index(drop=True)[:10],\
          data_amount['EPS - Net Income - Diluted'][::-1].reset_index(drop=True)[:10]

    try:
        work_table['eps'] = data_amount['EPS - Net Income - Diluted'][::-1].reset_index(drop=True)[:10]
    except:
        work_table['eps'] = data_amount['EPS - Net Income - Basic'][::-1].reset_index(drop=True)[:10]

    #Для расчетов за 11 год назад

    balance = pd.DataFrame()
    balance['date'] = main_data['reportdate'].T.drop_duplicates().T[::-1].reset_index(drop=True)[:11]

    balance['totalAssets'], balance['stockholdersEquity'] \
        = data_amount['Total Assets'][::-1].reset_index(drop=True)[:11], data_amount['''Stockholders' Equity'''][::-1].reset_index(drop=True)[:11] 


    work_table['grossProfit'] = (data_amount['Gross Margin'][::-1].reset_index(drop=True)[:10] * data_amount['Total Revenue'][::-1].reset_index(drop=True)[:10]) / 100

    work_table['totalDebt'] = (data_amount['Total Debt to Equity'][::-1].reset_index(drop=True)[:10] * data_amount['''Stockholders' Equity'''][::-1].reset_index(drop=True)[:10])

    work_table['longTermObligations'] = (data_amount['Long Term Debt to Equity'][::-1].reset_index(drop=True)[:10] * data_amount['''Stockholders' Equity'''][::-1].reset_index(drop=True)[:10])

    work_table['netDebt'] = work_table['currentLiabilities'] + work_table['totalDebt'] - work_table['currentAssets']


    work_table['incomeTaxExpense'] = work_table['netIncome'] * (data_amount['Calculated Tax Rate'][::-1].reset_index(drop=True)[:10] / 100)

    work_table['incomeBeforeTax'] = work_table['netIncome'] + work_table['incomeTaxExpense']

    work_table['freeCashFlow'] = data_amount['Free Cash Flow Per Share'][::-1].reset_index(drop=True)[:10] * data_amount['Weighted Average Shares Outstanding - Diluted'][::-1].reset_index(drop=True)[:10] 


    year_len = len(work_table.index) - 1
    # средние по сектору
    sector_table_pkl = pd.read_pickle("sector_margin_qandl.pkl")
    sector_table = pd.DataFrame(sector_table_pkl)
    
    return work_table, balance, sector_table, company_sector_ticker

In [31]:
def score(COUNTRY, COMPANY):
    data_ticker = data_csv[['ticker', 'compnumber', 'mic']]
    
    Total_rating = []
    
    work_table, balance, sector_table, company_sector_ticker = main_table_qan(COUNTRY, COMPANY) # загружаем таблицу
    
    
    year_len = len(work_table.index) - 1
    
#     # Получаем ставку облигации , курс валют , индекс

#     session = HTMLSession()
#     oblig_site = session.get('https://tradingeconomics.com/bonds')
#     oblig_pars = oblig_site.html.find('#p')

#     # ДОПИТСАТЬ ИНДЕКСЫ ДЛЯ СТРАН
#     COUNTRY_dict = {"DEU":[oblig_pars[3].text, wb.DataReader('EURUSD=X','yahoo')['Adj Close'][-1], '^GDAXI'],\
#                     "CAN": [oblig_pars[31].text, wb.DataReader('CADUSD=X','yahoo')['Adj Close'][-1], '^GSPTSE'],\
#                     "GBR": [oblig_pars[1].text, wb.DataReader('GBPUSD=X','yahoo')['Adj Close'][-1], '^FTSE'],\
#                     "KOR": [oblig_pars[55].text, wb.DataReader('KRWUSD=X','yahoo')['Adj Close'][-1], '^KS11'],\
#                     "HKG": [oblig_pars[65].text, wb.DataReader('HKDUSD=X','yahoo')['Adj Close'][-1], '^HSI'],\
#                     "SGP": [oblig_pars[57].text, wb.DataReader('SGDUSD=X','yahoo')['Adj Close'][-1], ''], \
#                     "USA": [oblig_pars[0].text, 1, ''], \
#                    }
#     COUNTRY_DF = pd.DataFrame(COUNTRY_dict)
#     course_exchange = COUNTRY_DF[data_ticker.loc[COUNTRY].index[0]][1]
#     INDEX = COUNTRY_DF[data_ticker.loc[COUNTRY].index[0]][2]
    
    mean_cagr = ((-work_table['revenue'][year_len] + work_table['revenue'][0])/abs(work_table['revenue'][year_len]))/year_len
    cagr_net_income = ((-work_table['netIncome'][year_len] + work_table['netIncome'][0])/abs(work_table['netIncome'][year_len]))/year_len 

#     # ________________________________________  Расчет WACC  _______________________________________________
#     start = datetime.datetime(2019, 2, 1)
#     end= datetime.datetime.today().strftime('%Y-%m-%d')

#     RF = float(COUNTRY_DF[data_ticker.loc[COUNTRY].index[0]][0]) / 100
# #     print(f'RF {RF}')

#     # Парсим beta с yahoo finance
#     pars_data_beta = yf.Ticker(COMPANY.replace(' ', '')) #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#     beta = pars_data_beta.info['beta']  
# #     print(f'beta {beta}')

#     # Считаем годовую доходность
#     data = wb.DataReader(INDEX, data_source='yahoo', start='2001-1-1') #  получаем индекс в начале скрипта
#     data['simple_return'] = (data['Adj Close'] / data['Adj Close'].shift(1)) - 1
#     data['simple_return'].replace(np.nan, 0)
#     avg_returns_a = data['simple_return'].mean() * 250

# #     print(f'Годовая доходность S&P500 {avg_returns_a*100}')


#     ke = RF + (beta * (avg_returns_a - RF))
# #     print(f'Стоимость капитала {ke*100}')

#     pars_mktCap = yf.Ticker(COMPANY.replace(' ', '')) #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#     marketCap = pars_data_beta.info['marketCap'] / course_exchange  # перевод валюты



#     WACC = ((marketCap / 1.22) / ((marketCap / 1.22) + work_table['netDebt'][0:2].mean())) * ke + (
#                     work_table['netDebt'][0:2].mean() / ((marketCap / 1.22) + work_table['netDebt'][0:2].mean())) * (
#                             work_table['interestExpense'][0] / work_table['netDebt'][0:2].mean()) \
#                    * (1 - (work_table['incomeTaxExpense'][0] / work_table['incomeBeforeTax'][0]))

#     print(f'WACC {WACC}')
#     print(f'KE {ke}')
#     print(f'mean_cagr {mean_cagr}')
#     print(f'cagr_net_income {cagr_net_income}')
    

# считаем рост выручки______________________________________________________________

    pr_revenue_growth = -work_table['revenue'][len(work_table['revenue']) - 1] + work_table['revenue'][0]

    revenue_growth_DMITR = (pr_revenue_growth / abs(work_table['revenue'][len(work_table['revenue']) - 1]) * 100) / len(work_table['revenue'])

    print(f'revenue_growth_DMITR {revenue_growth_DMITR}')

    if revenue_growth_DMITR >= 5:
        revenue_growth_rating = 1
    elif 5 > revenue_growth_DMITR >= 2:
        revenue_growth_rating = 0
    else:
        revenue_growth_rating = -1

    Total_rating.append(revenue_growth_rating)

    # считаем Использование прибыли______________________________________________________________
    SHAREHOLDERS_EQUITY = (work_table['totalAssets'] - work_table['totalLiabilities'])

    pr_SHAREHOLDERS_EQUITY = - SHAREHOLDERS_EQUITY[len(work_table['totalAssets']) -1 ] + SHAREHOLDERS_EQUITY[0]
    SHAREHOLDERS_EQUITY_growth_DMITR = (pr_SHAREHOLDERS_EQUITY / abs(SHAREHOLDERS_EQUITY[len(work_table['totalAssets']) -1 ]) * 100) / len(work_table['totalAssets'])


    use_of_profit_DMITR = round(SHAREHOLDERS_EQUITY_growth_DMITR, 2)

    print(f'use_of_profit_DMITR {use_of_profit_DMITR}')

    if use_of_profit_DMITR < 3:  
        use_of_profit_rating = -2
    elif 6 >= use_of_profit_DMITR >= 3:
        use_of_profit_rating = 0
    else:
        use_of_profit_rating = 2

    Total_rating.append(use_of_profit_rating)


    # Считаем маржу Валовой прибыли_________________________________________________________________


    GROSS_MAR = work_table['grossProfit']/work_table['revenue']

    gross_margin = round(np.mean(GROSS_MAR[0:]), 2) 

    print('*'*50)
    print(f'gross_margin {gross_margin}')

    gross_Margin_sector = pd.DataFrame(sector_table[company_sector_ticker]['Operating Margin'])
    qattile_gross_Margin = gross_Margin_sector.describe()

    if gross_margin <= qattile_gross_Margin.loc['25%'][0]:
        gross_margin_ratingz = -1

    elif gross_margin >= qattile_gross_Margin.loc['75%'][0]:
        gross_margin_ratingz = 1

    else:
        gross_margin_ratingz = 0

    Total_rating.append(gross_margin_ratingz)


    # Считаем Динамику валовой маржи_________________________________________________________________

    if GROSS_MAR.std() < 0.1:
        gross_margin_dynamics = 'Рост'
        gross_margin_dynamics_rating = 1
    else:
        gross_margin_dynamics = 'Падение'
        gross_margin_dynamics_rating = 0

    Total_rating.append(gross_margin_dynamics_rating)

    print(f'operating_margin_dynamics {gross_margin_dynamics}')    


    # Считаем Операционную маржу_________________________________________________________________

    OPERATING_MAR = work_table['operatingIncome']/work_table['revenue']

    operating_margin = round(np.mean(OPERATING_MAR[0:]), 2) 

    print(f'operating_margin {operating_margin}')                                                           


    gross_Margin_sector = pd.DataFrame(sector_table[company_sector_ticker]['Operating Margin'])
    qattile_operating_Margin = gross_Margin_sector.describe()                                                                   


    if operating_margin <= qattile_operating_Margin.loc['25%'][0]:
        operating_margin_rating = -1

    elif operating_margin >= qattile_operating_Margin.loc['75%'][0]:
        operating_margin_rating = 1

    else:
        operating_margin_rating = 0

    Total_rating.append(operating_margin_rating)

    # Считаем Динамику операционной маржи_________________________________________________________________

    if OPERATING_MAR.std() < 0.1:
        operating_margin_dynamics = 'Рост'
        operating_margin_dynamics_rating = 1
    else:
        operating_margin_dynamics = 'Падение'
        operating_margin_dynamics_rating = 0

    Total_rating.append(operating_margin_dynamics_rating)

    print(f'operating_margin_dynamics {operating_margin_dynamics}')

    # Считаем Маржу чистой прибыли_________________________________________________________________

    NET_PROFIT_MAR = work_table['netIncome']/work_table['revenue']

    net_profit_margin = round(np.mean(NET_PROFIT_MAR[0:]), 2)

    print(f'net_profit_margin {net_profit_margin}')

    net_Profit_Margin_sector = pd.DataFrame(sector_table[company_sector_ticker]['Operating Margin'])
    qattile_net_Profit_Margin = net_Profit_Margin_sector.describe() 

    if net_profit_margin <= qattile_net_Profit_Margin.loc['25%'][0]:
        net_profit_margin_rating = -1
    elif net_profit_margin >= qattile_net_Profit_Margin.loc['75%'][0]:
        net_profit_margin_rating = 1
    else:
        net_profit_margin_rating = 0

    Total_rating.append(net_profit_margin_rating)

    # Считаем Динамику маржи чистой прибыли_________________________________________________________________

    if NET_PROFIT_MAR.std() < 0.1 :
        net_profit_margin_dynamics = 'Рост'
        net_profit_margin_dynamics_rating = 1
    else:
        net_profit_margin_dynamics = 'Падение'
        net_profit_margin_dynamics_rating = 0

    Total_rating.append(net_profit_margin_dynamics_rating)

    print(f'net_profit_margin_dynamics {net_profit_margin_dynamics}')
    print('*'*50)

    # Считаем Рост EPS_________________________________________________________________

    pr_EPS_growth = -work_table['eps'][len(work_table['eps']) - 1] + work_table['eps'][0]
    EPS_growth_DMITR = (pr_EPS_growth / abs(work_table['eps'][len(work_table['eps']) - 1]) * 100) / len(work_table['eps'])


    print(f'EPS_growth_DMITR {EPS_growth_DMITR}')

    if EPS_growth_DMITR < 0.06:
        EPS_growth_rating = -2
    elif 0.1 >= EPS_growth_DMITR >= 0.06:
        EPS_growth_rating = 0
    else:
        EPS_growth_rating = 2

    Total_rating.append(EPS_growth_rating)

    # Считаем Уровень долга_________________________________________________________________

    debt_level = round(np.mean(work_table['totalLiabilities']/work_table['totalAssets']), 2)

    print(f'debt_level {debt_level}')

    if debt_level < 0.5:
        debt_level_rating = 1
    elif 0.7 >= debt_level >= 0.5:
        debt_level_rating = 0
    else:
        debt_level_rating = -1

    Total_rating.append(debt_level_rating)

     # ___________________________________ ROE_DuPont _______________________________________________

    Profitability = []
    TechnicalEfficiency = []
    FinancialStructure = []

    for year in range(0, 5):
        last_year = year + 1
        Profitability.append(work_table['netIncome'][year] / work_table['revenue'][year])
        TechnicalEfficiency.append(work_table['revenue'][year] / (
                (work_table['totalAssets'][year] + balance['totalAssets'].replace(np.nan, 0)[last_year]) / 2))
        FinancialStructure.append(((work_table['totalAssets'][year] + balance['totalAssets'].replace(np.nan, 0)[last_year]) / 2) /
                                          ((work_table['stockholdersEquity'][year] +
                                            balance['stockholdersEquity'].replace(np.nan, 0)[last_year]) / 2))

    table = pd.DataFrame()
    table['Profitability'] = Profitability
    table['TechnicalEfficiency'] = TechnicalEfficiency
    table['FinancialStructure'] = FinancialStructure

    work_table['ROE_DuPont'] = table['FinancialStructure'] * table['TechnicalEfficiency'] * \
                                       table['Profitability']

    ROE_DuPont = round(work_table['ROE_DuPont'].mean(axis=0), 5)


    print(f'ROE_DuPont_average {ROE_DuPont}')
    # print(decomp)

    ROE_DuPont_sector = pd.DataFrame(sector_table[company_sector_ticker]['ROE DuPont'])
    qattile_ROE_DuPont_sector = ROE_DuPont_sector.describe() 

    if ROE_DuPont <= qattile_ROE_DuPont_sector.loc['25%'][0]:
        ROE_DuPont_rating = -1
    elif ROE_DuPont >= qattile_ROE_DuPont_sector.loc['75%'][0]:
        ROE_DuPont_rating = 1
    else:
        ROE_DuPont_rating = 0

    Total_rating.append(ROE_DuPont_rating)

    # Считаем ROA_________________________________________________________________

    ROA = round(np.mean(work_table['netIncome']/work_table['totalAssets']), 2)

    print(f'ROA {ROA}')


    ROA_sector = pd.DataFrame(sector_table[company_sector_ticker]['ROA'])
    qattile_ROA_sector = ROA_sector.describe() 

    if ROA <= qattile_ROA_sector.loc['25%'][0]:
        ROA_rating = -1
    elif ROA >= qattile_ROA_sector.loc['75%'][0]:
        ROA_rating = 1
    else:
        ROA_rating = 0


    Total_rating.append(ROA_rating)



     # ___________________________________ ROCE _______________________________________________

    try:
        ROCE_10_year = []

        for j in range(0, year_len+1):
            ROCE = work_table['operatingIncome'][j] / (
                    (work_table['stockholdersEquity'][j] + work_table['longTermObligations'][j]) \
                    + (work_table['stockholdersEquity'][j] + work_table['longTermObligations'][j]) * 0.5)
            ROCE_10_year.append(ROCE)

        ROCE = np.mean(ROCE_10_year)
    except:
        pass

    print(f'ROCE = {ROCE}')


    ROCE_sector = pd.DataFrame(sector_table[company_sector_ticker]['ROCE'])
    qattile_ROCE_sector = ROCE_sector.describe() 

    if ROCE <= qattile_ROCE_sector.loc['25%'][0]:
        ROCE_rating = -1
    elif ROCE >= qattile_ROCE_sector.loc['75%'][0]:
        ROCE_rating = 1
    else:
        ROCE_rating = 0

    Total_rating.append(ROCE_rating)



#     # ROCE сравниваем с KE если больше KE на 2% то +1 если меньше -1 если на уровне 0 

#     if ROCE > ke + 0.02:
#         ROCE_KE_rating = 1
#     elif ROCE + 0.02 < ke:
#         ROCE_KE_rating = -1
#     else:
#         ROCE_KE_rating = 0

#     Total_rating.append(ROCE_KE_rating)
#     print(f'ke = {ke}')
    # Получаем общую оценку_________________________________________________________________

#     print(Total_rating)
#     print('*'*50)
#     print(f'Общая оценка = {sum(Total_rating)}')
#     print('*'*50)
#     # print(f'''Среднее по сектору = {median_by_Industry[sector_company[0]]}''')
#     print('*'*50)


    # -------------------------------Piotroski F-score----------------------------------

    cashflow_op = np.sum(work_table['operatingCashFlow'][0:3])

    ROA_FS = (round(work_table['netIncome'][0]/((work_table['totalAssets'][0] + work_table['totalAssets'][1])/2), 2)>0)  #  >0
    CFO_FS = int(work_table['operatingCashFlow'][0]>0)
    ROA_D_FS = int((round(work_table['netIncome'][0]/((work_table['totalAssets'][0] + work_table['totalAssets'][1])/2), 2))>
                   (round(work_table['netIncome'][1]/((work_table['totalAssets'][1] + work_table['totalAssets'][2])/2), 2)))
    CFO_ROA_FS = int((cashflow_op/work_table['totalAssets'][0])>(work_table['netIncome'][0]/
                        ((work_table['totalAssets'][0] + work_table['totalAssets'][1])/2)))          
    CR_FS = int((work_table['totalAssets'][0]/work_table['totalLiabilities'][0])>      
                 (work_table['totalAssets'][1]/work_table['totalLiabilities'][1]))           
    LTD_FS = int(work_table['longTermObligations'][0] <= work_table['longTermObligations'][1])
    DILUTION_FS = int(work_table['shares'][0] <= work_table['shares'][1])
    GM_FS = int(work_table['grossProfit'][0]/work_table['revenue'][0]>work_table['grossProfit'][1]/work_table['revenue'][1])
    ATO_FS = int((work_table['revenue'][0]/((work_table['totalAssets'][0] + work_table['totalAssets'][1])/2)) >
                (work_table['revenue'][1]/((work_table['totalAssets'][1] + work_table['totalAssets'][2]/2))))
    F_score = ROA_FS + CFO_FS + ROA_D_FS + CFO_ROA_FS + LTD_FS + CR_FS + DILUTION_FS + GM_FS + ATO_FS  



    # Aggregate_score = F_score * DSI # Нет данных по дивам

    # print(f'DSI is {DSI}') # Нет данных по дивам
#     print(f'F-score: {F_score}')

    # print(f'Aggregate_score is {Aggregate_score}') # Нет данных по дивам
    return sum(Total_rating), F_score

# Фильтр

In [32]:
if FILTER == 'sector' : 
    print('ФИЛЬТРУЕМ ПО СЕКТОРУ')
    data_csv_ticker = pd.read_csv('TICKER_companies.csv')
    country_filter = data_csv_ticker.set_index('countrycode')
    data_csv = data_csv_ticker.set_index('country')
    data_industry_sector = pd.read_excel('SECTORS.xlsx')

    # кол-во компаний:
    len_country = (len(data_csv_ticker.set_index('countrycode')))

    country_filter.fillna('нет тикера', inplace=True)

    listus =[]
    for i in range(len(country_filter.index)):
        listus.append(str(country_filter['mic'][i])[0:3])
    country_filter['mic_new'] = listus

    Industry_mic = data_industry_sector.set_index('Industry_mic') 

    sector_list = []
    for j in range(len(country_filter.index)):
        try:
            sector_list.append(Industry_mic.loc[float(country_filter['mic_new'][j])][0])
        except:
            sector_list.append('Oil & Gas')
                
    country_filter['sector'] = sector_list
    country_filter.set_index('sector')
    
    country_filter_sector = country_filter.set_index('sector')
    country_filter_sector.loc[SECTOR]
    
    SECTOR_Sheets = country_filter_sector.loc[SECTOR].index   
    TICKER_FOR = country_filter_sector.loc[SECTOR]['ticker']
    COUNTRY_Sheets = country_filter_sector.loc[SECTOR]['country'] 
    COMPANY_Name = country_filter_sector.loc[SECTOR]['shortname']

elif FILTER == 'country':
    data_csv_ticker = pd.read_csv('TICKER_companies.csv')
    print('ФИЛЬТРУЕМ ПО СТРАНЕ')
    data_csv = data_csv_ticker.set_index('country')
    data_industry_sector = pd.read_excel('SECTORS.xlsx')

    # Фильтр по стране:
    country_filter = data_csv_ticker.set_index('countrycode').loc[COUNTRY]
    # кол-во компаний:
    len_country = (len(data_csv_ticker.set_index('countrycode').loc[COUNTRY]))


    listus =[]
    for i in range(len(country_filter.index)):
        listus.append(str(country_filter['mic'][i])[0:3])
    country_filter['mic_new'] = listus

    Industry_mic = data_industry_sector.set_index('Industry_mic') 

    sector_list = []
    for i in range(len(country_filter.index)):
        try:
            sector_list.append(Industry_mic.loc[float(country_filter['mic_new'][j])][0])
        except:
            sector_list.append('Agriculture')

    country_filter['mic_new'] = listus
    country_filter['sector'] = sector_list
    # Собраная таблица со всеми данными
    country_filter.fillna('нет тикера', inplace=True)
    country_filter.set_index('sector')#.loc[COUNTRY]
    country_filter.fillna('0')
    
    TICKER = country_filter['ticker']
    TICKER_FOR = country_filter['ticker']
    SECTOR_Sheets = country_filter['sector']
    COUNTRY_Sheets = country_filter.index
    COMPANY_Name = country_filter['shortname']

#     TICKERS_FOR = country_filter_sector.loc[SECTOR]['ticker']

# Tickers_list = []
# SECTOR_list = []



TICKER_FOR
SECTOR_Sheets
COUNTRY_Sheets

ФИЛЬТРУЕМ ПО СЕКТОРУ


sector
Metals & Mining    USA
Metals & Mining    USA
Metals & Mining    USA
Metals & Mining    USA
Metals & Mining    USA
                  ... 
Metals & Mining    AUS
Metals & Mining    AUS
Metals & Mining    AUS
Metals & Mining    AUS
Metals & Mining    AUS
Name: country, Length: 1015, dtype: object

In [33]:
# len(TICKER_FOR.index)

for i in range(len(TICKER_FOR.index)):
    try:
        Total_rating, F_score = score(COUNTRY_Sheets[i], TICKER_FOR[i])
    except Exception as inst:
#         print(f'ошибка: {inst}')
        Total_rating, F_score = f'ошибка: {inst}', f'ошибка: {inst}'
        pass
    sleep(1)
    values = service.spreadsheets().values().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body={
            "valueInputOption": "USER_ENTERED",
            "data": [
                {"range": f"SECTORS!A{i+2}:G{i+2}",
                 "majorDimension": "ROWS",
                 "values": [
                     [COUNTRY_Sheets[i], TICKER_FOR[i], COMPANY_Name[i], SECTOR_Sheets[i],
                      str(Total_rating), str(F_score), 
                     ],
                 ]},
            ]
        }
    ).execute()
    



revenue_growth_DMITR 5.0380078636959365
use_of_profit_DMITR 9.43
**************************************************
gross_margin 0.19
operating_margin_dynamics Рост
operating_margin 0.08
operating_margin_dynamics Рост
net_profit_margin 0.04
net_profit_margin_dynamics Рост
**************************************************
EPS_growth_DMITR 3.0196078431372553
debt_level 0.82
ROE_DuPont_average 0.1518
ROA 0.04
ROCE = 0.09218727519425556
revenue_growth_DMITR 3.0207151811832125
use_of_profit_DMITR 8.88
**************************************************
gross_margin 0.17
operating_margin_dynamics Рост
operating_margin 0.07
operating_margin_dynamics Рост
net_profit_margin 0.04
net_profit_margin_dynamics Рост
**************************************************
EPS_growth_DMITR -9.874213836477988
debt_level 0.56
ROE_DuPont_average 0.06053
ROA 0.03
ROCE = 0.054682498825999216
revenue_growth_DMITR -6.246355659473386
use_of_profit_DMITR -10.91
**************************************************
gros

KeyboardInterrupt: 

DEU ALCO Agriculture